In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# Configuración de Chrome y Selenium
options = Options()
options.add_argument('--headless')  # Ejecuta Chrome en segundo plano
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Iniciar el navegador con el controlador Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1')

# Espera para que se carguen los comentarios dinámicos
time.sleep(5)

# Extraer todos los comentarios
comments = []
comment_elements = driver.find_elements(By.CLASS_NAME, 'ui-review-capability-comments__comment__content')
for element in comment_elements:
    comments.append(element.text)

# Cerrar el navegador
driver.quit()

# Imprimir los comentarios extraídos
for idx, comment in enumerate(comments, 1):
    print(f"Comentario {idx}: {comment}")

Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.


In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the MercadoLibre product
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'

# Send HTTP request
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Broaden search to capture all potential comment sections
comments = []
all_divs = soup.find_all('div')  # Finds all div elements to ensure no content is missed

# Extract text from each div and add it to comments if it's relevant
for div in all_divs:
    if div.get_text(strip=True):  # Only add non-empty text
        comments.append(div.get_text(strip=True))

# Check for the specific comment
found_comment = any("Hermoso tiene una super cama." in c for c in comments)

# Print results
if found_comment:
    print("The comment 'Hermoso tiene una super cama.' was successfully found.")
else:
    print("The comment 'Hermoso tiene una super cama.' was not found.")

# Print all extracted comments for verification
for idx, comment in enumerate(comments, 1):
    print(f"Comentario {idx}: {comment}")

The comment 'Hermoso tiene una super cama.' was not found.
Comentario 1: Saltar al contenidoComentar sobre accesibilidadMercado Libre Colombia - Donde comprar y vender de todoIngresa lo que quieras encontrarIngresa tuubicaciónIngresa tuubicaciónCategoríasVehículosSupermercadoTecnologíaElectrodomésticosHogar y MueblesDeportes y FitnessBelleza y Cuidado PersonalAccesorios para VehículosHerramientasConstrucciónInmueblesCompra InternacionalModaJuegos y JuguetesBebésProductos SustentablesMás vendidosSalud y Equipamiento MédicoIndustrias y OficinasServiciosTiendas oficialesVer más categoríasTecnologíaCelulares y TeléfonosCelulares y SmartphonesAccesorios para CelularesSmartwatchesComputaciónTabletsPortátilesComputadoresAccesorios para PortátilesImpresorasConectividad y RedesCámaras y AccesoriosCámarasAccesorios para CámarasInstrumentos ÓpticosElectrónica, Audio y VideoAccesorios Audio y VideoEquipos de DJ y AccesoriosDrones y AccesoriosAudioComponentes ElectrónicosConsolas y VideojuegosPara 

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Locate all comments with the specific class
comments = []
all_comments = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
for comment in all_comments:
    comments.append(comment.get_text(strip=True))

# Print extracted comments
for idx, comment in enumerate(comments, 1):
    print(f"Comentario {idx}: {comment}")

Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Set up Selenium with Chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (no GUI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the MercadoLibre product page
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'
driver.get(url)

# Wait for the page to load dynamic content
time.sleep(5)

# Scroll down to load more comments if necessary (repeat scrolling if there are "Load more" buttons)
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new comments to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Extract all comments
comments = []
review_elements = driver.find_elements(By.CLASS_NAME, 'ui-review-capability-comments__comment__content')
for element in review_elements:
    comments.append(element.text)

# Close the browser
driver.quit()

# Store comments in a DataFrame
reviews_df = pd.DataFrame(comments, columns=['reviews'])

# Display the DataFrame
print(reviews_df)

                                             reviews
0  Está bueno y funciona bien, esperemos que no s...
1  Me parece un movil muy bonito, tiene un duraci...


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Set up Selenium with Chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the MercadoLibre product page
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'
driver.get(url)
time.sleep(5)

# Scroll and attempt to click "Load More" if present
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # wait for the new content to load
    
    try:
        # Attempt to click "Load More" button to expand all comments
        load_more_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Mostrar más')]")
        load_more_button.click()
        time.sleep(2)  # wait for comments to load
    except:
        pass

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Extract all comments
comments = []
review_elements = driver.find_elements(By.CLASS_NAME, 'ui-review-capability-comments__comment__content')
for element in review_elements:
    comments.append(element.text)

# Close the browser
driver.quit()

# Store comments in a DataFrame
reviews_df = pd.DataFrame(comments, columns=['reviews'])

# Display the DataFrame
print(reviews_df)

                                             reviews
0  Está bueno y funciona bien, esperemos que no s...
1  Me parece un movil muy bonito, tiene un duraci...


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import time

# Configura el controlador de Chrome con Service
driver_path = 'D:\Documents\Scrapping Sentimientos Project\Project_repository\\chromedriver.exe'  # Ruta completa a tu ChromeDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# URL de la página
url = 'https://www.mercadolibre.com.co/xiaomi-redmi-note-13-pro-4g-dual-sim-256-gb-purpura-8-gb-ram/p/MCO35734320?pdp_filters=deal%3AMCO1072578-1'
driver.get(url)

# Espera inicial para cargar la página
time.sleep(25)

# Paso 1: Haz clic en el botón "Mostrar todas las opiniones" para abrir la ventana emergente
try:
    load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.show-more-click[data-testid="see-more"]')
    driver.execute_script("arguments[0].scrollIntoView();", load_more_button)  # Desplaza hasta el botón
    time.sleep(1)
    driver.execute_script("arguments[0].click();", load_more_button)  # Haz clic en el botón
    time.sleep(3)  # Espera para que la ventana emergente de comentarios se abra
except (NoSuchElementException, ElementClickInterceptedException) as e:
    print("No se encontró o no se pudo hacer clic en el botón de 'Mostrar todas las opiniones'.", e)

# Paso 2: Encuentra el contenedor específico de comentarios
try:
    comments_container = driver.find_element(By.CSS_SELECTOR, 'section[data-testid="reviews-desktop"]')
except NoSuchElementException:
    print("No se encontró el contenedor de comentarios.")
    driver.quit()
    exit()

# Paso 3: Desplazamiento progresivo dentro del contenedor de comentarios para cargar todos los bloques de comentarios
scroll_pause_time = 2
last_height = driver.execute_script("return arguments[0].scrollHeight", comments_container)

while True:
    # Desplaza dentro del contenedor de comentarios
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_container)
    time.sleep(scroll_pause_time)

    # Calcula la nueva altura de desplazamiento del contenedor
    new_height = driver.execute_script("return arguments[0].scrollHeight", comments_container)
    if new_height == last_height:
        # Si la altura no cambia, ya no hay más comentarios para cargar
        break
    last_height = new_height

# Paso 4: Extrae los comentarios de la página completamente cargada con BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
comment_elements = soup.find_all('p', class_='ui-review-capability-comments__comment__content')
comments = [comment.get_text(strip=True) for comment in comment_elements]

# Imprime los comentarios extraídos
for idx, comment in enumerate(comments, 1):
    print(f"Comentario {idx}: {comment}")

# Cierra el navegador
driver.quit()

Comentario 1: Está bueno y funciona bien, esperemos que no salga chimbo, como mi ex.
Comentario 2: Me parece un movil muy bonito, tiene un duracion de bateria excelente.
